### **🧠 Understanding Function Calling with OpenAI API (Function Tool Calling)**

In this lesson, we explore **Function Calling** with OpenAI's API — a powerful feature that allows the model to call specific functions based on user intent. This makes the model useful for building **AI agents**, **chatbots**, and **smart automation systems**.

### **🔧 What is Function Calling?**

Function calling lets the AI decide **when** and **how** to call an external function you've defined — based on the user's query. You define the function's name, parameters, and descriptions, and the model takes care of the rest.

### **🚀 New in this Notebook: Multi-Function Calling**

A standout feature of the OpenAI API is its ability to execute **multiple function calls** in response to a single user query. For instance, if a user asks, *"What's the stock price of Google and what time is it in Tokyo?"*, the model can intelligently decide to call both the `get_stock_price` and `get_current_time` functions simultaneously. 

This notebook has been updated to demonstrate this multi-function calling capability. We will define a comprehensive set of tools and then feed the model complex queries that require multiple tools to be called in parallel. The results from all function calls are then sent back to the model in a single step, allowing it to synthesize a complete answer.

### **Setup and Imports**

First, let's install the necessary libraries and set up our environment.

In [ ]:
!pip install openai python-dotenv pytz

### **Step 1: Define All Mock Functions**
First, we define all the individual Python functions that our AI can potentially call. These are mock functions for demonstration.

In [1]:
import os
import json
import random
import datetime
import pytz
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize the OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# --- Mock Functions ---

def convert_currency(amount, from_currency, to_currency):
    """Converts an amount from one currency to another."""
    rates = {"USD_to_EUR": 0.92, "EUR_to_USD": 1.09}
    key = f"{from_currency}_to_{to_currency}"
    if key in rates:
        converted_amount = amount * rates[key]
        return f"{amount} {from_currency} is equal to {converted_amount:.2f} {to_currency}."
    else:
        return f"Conversion from {from_currency} to {to_currency} is not supported."

def get_current_time(timezone):
    """Gets the current time in a specific timezone."""
    try:
        tz = pytz.timezone(timezone)
        current_time = datetime.datetime.now(tz)
        return f"The current time in {timezone} is {current_time.strftime('%H:%M:%S')}."
    except pytz.UnknownTimeZoneError:
        return f"Unknown timezone: {timezone}"

def get_stock_price(company_name):
    """Gets the current stock price for a company."""
    prices = {"Apple": "175.20 USD", "Google": "135.50 USD"}
    return prices.get(company_name, f"Stock price for {company_name} not found.")

def evaluate_math_expression(expression):
    """Evaluates a math expression."""
    try:
        result = eval(expression)
        return f"The result of '{expression}' is {result}."
    except:
        return "Invalid math expression."

def get_random_joke():
    """Fetches a random joke."""
    jokes = [
        "Why don't scientists trust atoms? Because they make up everything!",
        "Why did the scarecrow win an award? Because he was outstanding in his field!",
        "What do you call a lazy kangaroo? Pouch potato."
    ]
    return random.choice(jokes)

def get_capital_city(country):
    """Gets the capital city of a country."""
    capitals = {"Japan": "Tokyo", "France": "Paris", "Germany": "Berlin"}
    return capitals.get(country, f"Capital of {country} not found.")

def get_latest_news(topic):
    """Gets the latest news on a topic."""
    return f"Latest news on {topic}: AI models are becoming increasingly powerful."

def get_user_email(username):
    """Retrieves a user's email by username."""
    emails = {"john_doe": "john.doe@example.com"}
    return emails.get(username, f"Email for {username} not found.")

### **Step 2: Define All Tool-Specifications**
We now create a single `all_tools` list that contains the JSON schema for every function defined above. The model will use these schemas to understand what functions are available, what they do, and what parameters they accept.

In [2]:
all_tools = [
    {
        "type": "function",
        "function": {
            "name": "convert_currency",
            "description": "Convert an amount from one currency to another.",
            "parameters": {
                "type": "object",
                "properties": {
                    "amount": {"type": "number"},
                    "from_currency": {"type": "string"},
                    "to_currency": {"type": "string"}
                },
                "required": ["amount", "from_currency", "to_currency"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current time in a specific timezone.",
            "parameters": {
                "type": "object",
                "properties": {
                    "timezone": {
                        "type": "string",
                        "description": "The timezone, e.g., 'America/New_York' or 'Asia/Tokyo'."
                    }
                },
                "required": ["timezone"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": "Get the current stock price for a company.",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {"type": "string"}
                },
                "required": ["company_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "evaluate_math_expression",
            "description": "Evaluate a math expression.",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {"type": "string"}
                },
                "required": ["expression"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_random_joke",
            "description": "Get a random joke.",
            "parameters": {"type": "object", "properties": {}}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_capital_city",
            "description": "Get the capital city of a country.",
            "parameters": {
                "type": "object",
                "properties": {
                    "country": {"type": "string"}
                },
                "required": ["country"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_latest_news",
            "description": "Get the latest news on a topic.",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {"type": "string"}
                },
                "required": ["topic"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_user_email",
            "description": "Get a user's email by username.",
            "parameters": {
                "type": "object",
                "properties": {
                    "username": {"type": "string"}
                },
                "required": ["username"]
            }
        }
    }
]

### **Step 3: Create the Multi-Function Execution Flow**

This is the core logic. The `execute_multi_function_call` helper function handles the entire process:
1.  Sends the user's query and the list of all available tools to the model.
2.  Checks if the model's response contains `tool_calls`. This is a list of all the functions the model wants to run.
3.  **It iterates through this list.** For each tool call, it executes the corresponding Python function.
4.  It collects all the results from these function calls.
5.  It sends the original query, the model's decision to call functions, and the results of those functions back to the model in a second API call.
6.  The model then uses all this information to provide a final, coherent, natural-language answer.

In [5]:
def execute_multi_function_call(user_query, tools):
    """A helper function to execute the full multi-function calling flow."""
    messages = [{"role": "user", "content": user_query}]
    
    # First API call to the model
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    response_message = response.choices[0].message
    messages.append(response_message)  # Append the model's response
    
    # Check if the model wants to call one or more functions
    if response_message.tool_calls:
        available_functions = {
            "convert_currency": convert_currency,
            "get_current_time": get_current_time,
            "get_stock_price": get_stock_price,
            "evaluate_math_expression": evaluate_math_expression,
            "get_random_joke": get_random_joke,
            "get_capital_city": get_capital_city,
            "get_latest_news": get_latest_news,
            "get_user_email": get_user_email,
        }
        
        # Loop through each tool call requested by the model
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions.get(function_name)
            
            if function_to_call:
                function_args = json.loads(tool_call.function.arguments)
                # Handle functions with no arguments like get_random_joke
                if function_name == "get_random_joke":
                    function_response = function_to_call()
                else:
                    function_response = function_to_call(**function_args)
                
                # Send the function's result back to the model
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": function_response
                })

        # Second API call to get a natural language response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )
        return second_response.choices[0].message.content
    else:
        return response_message.content

### **Step 4: Execute Multi-Function Prompts**

Now, let's test our setup with a variety of prompts designed to trigger multiple function calls. We will loop through 30 different questions and observe how the model handles them.

In [6]:
multi_function_prompts = [
    # Two function calls
    "What is the stock price of Apple and what time is it in 'America/New_York'?",
    "Convert 100 USD to EUR and tell me a joke.",
    "What is the capital of Japan and what is the latest news on technology?",
    "Please calculate 25 * 8 and find the email for the user 'john_doe'.",
    "What time is it in London and what is the capital of France?",
    "Tell me the stock price for Google and a random joke.",
    "What is the latest news about AI and what is 5 to the power of 3?",
    "Convert 200 EUR to USD and what is the capital of Germany?",
    "What is the email for 'john_doe' and what time is it in 'Asia/Dubai'?",
    "Find the capital of Japan and tell me the stock price for Apple.",
    "Calculate (100+50)/2 and tell me a joke.",
    "What's the latest news on 'global politics' and what time is it in Moscow?",
    "Convert 50 EUR to USD and what is the stock price of Google?",
    "What is the capital of France and what is 1024 / 32?",
    "Tell me a joke and what time is it in 'Australia/Sydney'.",
    
    # Three function calls
    "What is the capital of Germany, the stock price of Google, and the current time in Berlin?",
    "Convert 150 USD to EUR, tell me a joke, and calculate 99/9.",
    "Tell me the latest news on 'healthcare', find the capital of Japan, and what is the stock price for Apple?",
    "What time is it in Tokyo, what is the capital of Japan, and can you tell me a joke?",
    "Calculate the result of '12 * 12', convert 100 EUR to USD, and what is the stock price of Apple?",
    "Find the user email for 'john_doe', the capital of France, and the time in Paris.",
    "What's the latest news on AI, the stock price for Google, and a good joke?",
    "What is 500-250, what's the time in 'America/Los_Angeles', and the capital of Japan?",
    "Convert 75 USD to EUR, what's the stock price of Apple, and tell me a joke.",
    "What's the capital of Germany, the latest news on 'finance', and what time is it in Berlin?",

    # Four function calls
    "Convert 100 USD to EUR, tell me the stock price for Apple, find the time in 'America/New_York', and then tell me a joke.",
    "What is the capital of Japan, the latest news on 'technology', the result of '32*4', and the stock price of Google?",
    "Find the email for 'john_doe', convert 100 EUR to USD, calculate '10+20+30', and what is the capital of France?",
    "Tell me a joke, the time in London, the stock price of Apple, and the latest news about 'AI'.",
    "What is the capital of Germany, convert 50 EUR to USD, what's the time in Berlin, and what is the result of 2^8?"
]

# Execute the flow for each prompt
for prompt in multi_function_prompts:
    print(f"👤 User: {prompt}")
    final_response = execute_multi_function_call(prompt, all_tools)
    print(f"🤖 AI: {final_response}")
    print("-" * 50)
    print()

👤 User: What is the stock price of Apple and what time is it in 'America/New_York'?
🤖 AI: The current stock price of Apple is 175.20 USD. The time in the America/New_York timezone is 13:24:11 (1:24 PM).
--------------------------------------------------

👤 User: Convert 100 USD to EUR and tell me a joke.
🤖 AI: 100 USD is equal to 92.00 EUR. And here's a joke for you: Why did the scarecrow win an award? Because he was outstanding in his field!
--------------------------------------------------

👤 User: What is the capital of Japan and what is the latest news on technology?
🤖 AI: The capital of Japan is Tokyo. In the realm of technology, a significant development is that AI models are becoming increasingly powerful.
--------------------------------------------------

👤 User: Please calculate 25 * 8 and find the email for the user 'john_doe'.
🤖 AI: The result of \(25 \times 8\) is 200, and the email for the user 'john_doe' is john.doe@example.com.
-----------------------------------------